# Quickstart Tutorial: Basic Custom Chatbot using Atomic Agents

This quickstart tutorial demonstrates how to create a custom chatbot with a unique personality using the Atomic Agents library. The chatbot will respond to user inputs in rhyming verse.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennyVaneetvelde/atomic_agents/blob/main/examples/notebooks/quickstart.ipynb#)


# Prerequisites

Before proceeding with this notebook, it is highly recommended to read up on the basics of the following libraries:

- **Pydantic**: A data validation and settings management library using Python type annotations. You can find more information and documentation at [Pydantic GitHub](https://github.com/pydantic/pydantic).
- **Instructor**: A Python library that simplifies working with structured outputs from large language models (LLMs). It provides a user-friendly API to manage validation, retries, and streaming responses. More details can be found at [Instructor GitHub](https://github.com/jxnl/instructor).

Understanding these libraries will help you make the most of this library.


## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install `atomic-agents`, `openai`, and `instructor` libraries.

In [ ]:
# Install necessary packages
%pip install atomic-agents openai instructor

## Step 2: Import Libraries

We will import the necessary libraries for creating the chatbot. Each library serves a specific purpose:
- `ChatMemory`: Manages the chat history.
- `BaseChatAgent`: The base class to create a custom chatbot. Can be extended for additional functionality if needed.
- `SystemPromptGenerator` and `SystemPromptInfo`: To define and generate system prompts.

In [ ]:
import os
import instructor
import openai
from atomic_agents.lib.components.chat_memory import ChatMemory
from atomic_agents.agents.base_chat_agent import BaseChatAgent, BaseChatAgentConfig
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator, SystemPromptInfo

## Step 3: Define System Prompt Information

We will define the system prompt information including background, steps, and output instructions. This helps the chatbot understand how to respond to user inputs.
In this example, we will define a system prompt that asks the chatbot to respond to user inputs in rhyming verse.

The structuring of the system prompts in this library is loosely inspired by Patterns in the [Fabric library](https://github.com/danielmiessler/fabric/tree/main/patterns).

In [ ]:
system_prompt = SystemPromptInfo(
    background=[
        'This assistant is a general-purpose AI designed to be helpful and friendly.',
    ],
    steps=[
        'Understand the user\'s input and provide a relevant response.',
        'Respond to the user.'
    ],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be friendly and respectful in all interactions.',
        'Always answer in rhyming verse.'
    ]
)
system_prompt_generator = SystemPromptGenerator(system_prompt)

from rich.markdown import Markdown
Markdown(system_prompt_generator.generate_prompt())

## Step 4: Initialize Chat Memory

We will initialize the chat memory to store conversation history and load an initial greeting message.

In [ ]:
memory = ChatMemory()
initial_memory = [
    {'role': 'assistant', 'content': 'How do you do? What can I do for you? Tell me, pray, what is your need today?'}
]
memory.load(initial_memory)

## Step 5: Set Up API Key and Create Chat Agent

To use the **OpenAI** API, you need to set up your API key. You can either enter it directly in the code or set it as an environment variable. Additionally, you can choose other clients such as **Anthropic**, **Mistral**, etc. For a full list, check out the [Instructor library docs](https://github.com/jxnl/instructor).

In [ ]:
##################################################################
# ENTER YOUR API KEY BELOW, OR SET IT AS AN ENVIRONMENT VARIABLE #
##################################################################
API_KEY = ''
if not API_KEY:
    # Get the environment variable
    API_KEY = os.getenv('OPENAI_API_KEY')

if not API_KEY:
    raise ValueError('API key is not set. Please set the API key as a static variable or in the environment variable OPENAI_API_KEY.')

client = instructor.from_openai(openai.OpenAI(api_key=API_KEY))

agent = BaseChatAgent(
    config=BaseChatAgentConfig(
        client=client,
        system_prompt_generator=system_prompt_generator,
        model='gpt-3.5-turbo',
        memory=memory,
    )
)

## Step 6: Main Chat Loop

We will create a main chat loop for testing the chat agent. This loop will allow you to interact with the chatbot in real-time.
To exit the loop, type `/exit` or `/quit`.

In [ ]:
print(f'Agent: {initial_memory[0]["content"]}')

while True:
    user_input = input('You: ')
    if user_input.lower() in ['/exit', '/quit']:
        print('Exiting chat...')
        break

    # Note: The .model_dump() method is ONLY necessary in an ipython notebook.
    # In a normal Python script, you can simply pass the input_schema() run() method like so:
    # response = agent.run(agent.input_schema(chat_input=user_input))
    # The reason for this is unknown to me and I have not been able to find any documentation on it.
    response = agent.run(agent.input_schema(chat_input=user_input).model_dump())
    print(f'Agent: {response.response}')